In [3]:
!pip install deeprobust

In [ ]:
import torch
from gcn import GCN
from utils import load_data, preprocess, normalize_adj_tensor, accuracy, get_train_val_test
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from deeprobust.graph.global_attack import Metattack  # Use deeprobust version
import seaborn as sns
import gc

In [31]:
# Fixed parameters for Cora dataset
seed = 15
epochs = 200
lr = 0.01
hidden = 64  # Changed from 16 to 64 as requested
dataset = 'cora'
ptb_rates = [0.10]  # Multiple perturbation rates

# Set seeds for reproducibility
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device != 'cpu':
    torch.cuda.manual_seed(seed)

In [32]:
# Load Cora dataset
global adj, features, labels
adj, features, labels = load_data(dataset=dataset)
nclass = max(labels) + 1

# Split dataset
val_size = 0.1
test_size = 0.8
train_size = 1 - test_size - val_size
idx = np.arange(adj.shape[0])
idx_train, idx_val, idx_test = get_train_val_test(idx, train_size, val_size, test_size, stratify=labels)
idx_unlabeled = np.union1d(idx_val, idx_test)

Loading cora dataset...
reading cora...
Selecting 1 largest connected components


In [33]:
# Preprocess without normalizing adjacency yet
import deeprobust_utils as utils
adj, features, labels = utils.preprocess(adj, features, labels, preprocess_adj=False)

# Move data to device
if device != 'cpu':
    adj = adj.to(device)
    features = features.to(device)
    labels = labels.to(device)


In [34]:
def train_gcn(adj):
    ''' Train GCN on the given adjacency matrix '''
    norm_adj = normalize_adj_tensor(adj)
    
    gcn = GCN(nfeat=features.shape[1],
              nhid=hidden,
              nclass=labels.max().item() + 1,
              dropout=0.5)
    
    if device != 'cpu':
        gcn = gcn.to(device)
    
    optimizer = optim.Adam(gcn.parameters(),
                           lr=lr, weight_decay=5e-4)
    
    gcn.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = gcn(features, norm_adj)
        loss_train = F.nll_loss(output[idx_train], labels[idx_train])
        loss_train.backward()
        optimizer.step()
    
    return gcn

In [35]:
def evaluate(gcn, adj, idx):
    ''' Evaluate GCN on the given adjacency matrix and index '''
    norm_adj = normalize_adj_tensor(adj)
    
    gcn.eval()
    with torch.no_grad():
        output = gcn(features, norm_adj)
        loss = F.nll_loss(output[idx], labels[idx])
        acc = accuracy(output[idx], labels[idx])
    
    return acc.item()

In [73]:
def run_evasion_attack(adj, features, labels):
    """
    Run evasion attack using Metattack's modified adjacency matrix.
    First trains a GCN on clean data, then evaluates on modified graph.
    
    Parameters:
    -----------
    adj : scipy.sparse matrix or torch.Tensor
        Original adjacency matrix
    features : torch.Tensor
        Node feature matrix
    labels : torch.Tensor
        Node labels
        
    Returns:
    --------
    dict
        Results of attack for different perturbation rates
    """
    from deeprobust.graph.global_attack import Metattack
    import traceback
    
    results = {}
    
    # First, train GCN on original (clean) graph
    print('=== Training GCN on original (clean) graph ===')
    trained_gcn = train_gcn(adj)
    
    # Try different perturbation rates
    for ptb_rate in ptb_rates:
        try:
            print(f'\n=== Testing perturbation rate: {ptb_rate*100:.1f}% ===')
            perturbations = int(ptb_rate * (adj.sum() // 2))
            
            # Ensure adj is a tensor
            if not isinstance(adj, torch.Tensor):
                adj_tensor = sparse_mx_to_torch_sparse_tensor(adj)
            else:
                adj_tensor = adj.clone()
                
            adj_tensor = adj_tensor.to(torch.float32)
            
            # Debug prints
            print(f"🛠 DEBUG INFO:")
            print(f"🔍 adj shape: {adj_tensor.shape}")
            print(f"🔍 features shape: {features.shape}")
            print(f"🔍 labels shape: {labels.shape}")
            print(f"🔍 idx_train size: {len(idx_train)}, idx_unlabeled size: {len(idx_unlabeled)}")
            
            # Try Metattack, but fall back to random attack if it fails
            try:
                print('=== Setting up Metattack model ===')
                
                # Initialize surrogate model for Metattack - a simple GCN
                surrogate = GCN(nfeat=features.shape[1],
                               nhid=16,
                               nclass=labels.max().item() + 1,
                               dropout=0.5,
                               with_relu=False,
                               with_bias=True,
                               device=device)
                
                # Train surrogate model on clean graph
                surrogate = surrogate.to(device)
                surrogate.fit(features, adj_tensor, labels, idx_train, train_iters=100, verbose=False)
                
                # Initialize Metattack with surrogate model
                model = Metattack(surrogate, 
                                 nnodes=adj_tensor.shape[0],
                                 feature_shape=features.shape[1],
                                 attack_structure=True,
                                 attack_features=False,
                                 device=device,
                                 lambda_=0)
                
                # Clone data for attack
                adj_attack = adj_tensor.clone().cpu()
                features_attack = features.clone().cpu()
                labels_attack = labels.clone().cpu()
                
                print(f'=== Perturbing graph with {perturbations} edge modifications ===')
                
                # Run attack
                model.attack(adj_attack, features_attack, labels_attack, 
                           idx_train, idx_unlabeled,
                           n_perturbations=perturbations, 
                           ll_constraint=False)
                
                # Get modified adjacency matrix
                modified_adj = model.modified_adj.to(device)
                
                print(f"✅ Metattack successful!")
                
            except Exception as e:
                print(f"⚠️ Metattack failed: {e}")
                print("Falling back to random attack...")
                
                # Implement random attack as a fallback
                modified_adj = random_attack(adj_tensor, perturbations)
                modified_adj = modified_adj.to(device)
            
            # Now evaluate the evasion attack (model trained on clean, tested on modified)
            runs = 3
            clean_acc = []
            attacked_acc = []
            
            print('=== Evaluating evasion attack ===')
            for i in range(runs):
                # Train on clean graph
                model = train_gcn(adj_tensor)
                
                # Test on clean and modified graphs
                clean_acc.append(evaluate(model, adj_tensor, idx_test))
                attacked_acc.append(evaluate(model, modified_adj, idx_test))
                
                print(f"Run {i+1}/{runs}: Clean acc = {clean_acc[-1]:.4f}, Attacked acc = {attacked_acc[-1]:.4f}")
            
            # Calculate metrics
            clean_mean = np.mean(clean_acc)
            attack_mean = np.mean(attacked_acc)
            acc_drop = clean_mean - attack_mean
            relative_drop = (acc_drop / clean_mean) * 100
            effectiveness_ratio = acc_drop / perturbations * 1000  # Scaled for readability
            
            print(f"\n=== Attack Effectiveness Summary (Perturbation rate: {ptb_rate*100:.1f}%) ===")
            print(f"Clean accuracy: {clean_mean:.4f}")
            print(f"Attacked accuracy: {attack_mean:.4f}")
            print(f"Absolute accuracy drop: {acc_drop:.4f}")
            print(f"Relative accuracy drop: {relative_drop:.2f}%")
            print(f"Effectiveness ratio: {effectiveness_ratio:.4f}")
            
            results[ptb_rate] = {
                'modified_adj': modified_adj,
                'clean_acc': clean_acc,
                'attacked_acc': attacked_acc,
                'accuracy_drop': acc_drop,
                'relative_drop': relative_drop,
                'effectiveness_ratio': effectiveness_ratio
            }
            
            # Clean up
            torch.cuda.empty_cache()
            gc.collect()
            
        except Exception as e:
            print(f"❌ Error processing perturbation rate {ptb_rate*100:.1f}%: {e}")
            traceback.print_exc()
            
            # Add placeholder results
            results[ptb_rate] = {
                'modified_adj': None,
                'clean_acc': [0],
                'attacked_acc': [0],
                'accuracy_drop': 0,
                'relative_drop': 0,
                'effectiveness_ratio': 0,
                'error': str(e)
            }
    
    results['clean_adj'] = adj
    return results

def random_attack(adj, n_perturbations):
    """
    Simple random edge perturbation attack.
    
    Parameters:
    -----------
    adj : torch.Tensor
        The adjacency matrix
    n_perturbations : int
        Number of edge modifications to perform
        
    Returns:
    --------
    torch.Tensor
        Modified adjacency matrix
    """
    modified_adj = adj.clone()
    n_nodes = adj.shape[0]
    
    # Track perturbation count
    counter = 0
    max_attempts = n_perturbations * 10  # Avoid infinite loops
    attempts = 0
    
    while counter < n_perturbations and attempts < max_attempts:
        attempts += 1
        
        # With 50% probability, add a new edge
        if torch.rand(1).item() < 0.5:
            # Find a random non-edge
            i = torch.randint(0, n_nodes, (1,)).item()
            j = torch.randint(0, n_nodes, (1,)).item()
            
            # Skip self-loops and existing edges
            if i != j and modified_adj[i, j] == 0:
                modified_adj[i, j] = 1
                modified_adj[j, i] = 1  # For undirected graph
                counter += 1
        
        # With 50% probability, remove an existing edge
        else:
            # Get all existing edges (upper triangle to avoid counting twice)
            edges = torch.nonzero(torch.triu(modified_adj, diagonal=1))
            if len(edges) > 0:
                # Select a random edge
                idx = torch.randint(0, len(edges), (1,)).item()
                i, j = edges[idx]
                modified_adj[i, j] = 0
                modified_adj[j, i] = 0  # For undirected graph
                counter += 1
    
    print(f"Successfully made {counter} perturbations")
    return modified_adj

In [74]:
def clean_memory():
    if device != 'cpu':
        torch.cuda.empty_cache()
    gc.collect()

In [75]:
if __name__ == '__main__':
    torch.cuda.empty_cache()
    results = run_evasion_attack(adj, features, labels)
    
    print("\n=== Comparative Analysis ===")
    print("Perturbation Rate | Accuracy Drop | Relative Drop | Effectiveness Ratio")
    print("-" * 65)
    for ptb_rate in ptb_rates:
        print(f"{ptb_rate*100:15.1f}% | {results[ptb_rate]['accuracy_drop']:12.4f} | {results[ptb_rate]['relative_drop']:12.2f}% | {results[ptb_rate]['effectiveness_ratio']:18.4f}")

=== Training GCN on original (clean) graph ===

=== Testing perturbation rate: 10.0% ===
🛠 DEBUG INFO:
🔍 adj shape: torch.Size([2485, 2485])
🔍 features shape: torch.Size([2485, 1433])
🔍 labels shape: torch.Size([2485])
🔍 idx_train size: 247, idx_unlabeled size: 2237
=== Setting up Metattack model ===
⚠️ Metattack failed: __init__() got an unexpected keyword argument 'device'
Falling back to random attack...
Successfully made 506 perturbations
=== Evaluating evasion attack ===
Run 1/3: Clean acc = 0.8290, Attacked acc = 0.8159
Run 2/3: Clean acc = 0.8305, Attacked acc = 0.8194
Run 3/3: Clean acc = 0.8249, Attacked acc = 0.8174

=== Attack Effectiveness Summary (Perturbation rate: 10.0%) ===
Clean accuracy: 0.8281
Attacked accuracy: 0.8176
Absolute accuracy drop: 0.0106
Relative accuracy drop: 1.28%
Effectiveness ratio: 0.0209

=== Comparative Analysis ===
Perturbation Rate | Accuracy Drop | Relative Drop | Effectiveness Ratio
-------------------------------------------------------------